<a href="https://colab.research.google.com/github/JonathanFSP/Projeto_Imersao_Alura/blob/main/Professor_IAn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
#Importação de bibliotecas
import pandas as pd
import numpy as np
import re

# Import the Python SDK
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get ('SECRET_KEY')
genai.configure(api_key=api_key)

In [3]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [4]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.2, #aleatoriedade das palavras de 0 à 1
}

# NONE, FEW, SOME, MOST
safety_settings = {
    "HARASSMENT": "block_none",
    "HATE": "block_none",
    "SEXUAL": "block_none",
    "DANGEROUS": "block_none"
}

#escolhendo e inicializando o modelo com as configurações
model = genai.GenerativeModel (model_name = "gemini-1.0-pro",
                                generation_config=generation_config,
                               safety_settings = safety_settings)


In [ ]:
from termcolor import colored

def conversar_com_especialista_IA():
    """
    Conversa com a IA como especialista em um tema específico.
    Returns:
        None. Imprime a conversa no console.
    """
    try:
        print(colored("\n-- Bem-vindo ao Chat, sou o professor IAn! --", "blue", attrs=["bold"]))
        print(colored("Olá! 👋 Sou seu especialista em IA. Vamos conversar sobre o que você quiser.\n", "cyan"))

        tema = input("Qual tema te interessa hoje? ")

        print(colored(f"\n## Tema: {tema.upper()} ##\n", "blue", attrs=["bold"]))
        print(colored("Qual o seu nível de conhecimento em {}?\n".format(tema), "cyan"))
        print(colored("1 - Iniciante (apenas começando) 👶", "yellow"))
        print(colored("2 - Intermediário (já tenho alguma base) 📚", "green"))
        print(colored("3 - Especialista (conheço o tema a fundo) 🤓", "blue"))

        while True:
            try:
                nivel = int(input("Escolha uma opção (1-3): "))
                if 1 <= nivel <= 3:
                    break
                else:
                    print(colored("Opção inválida. Escolha um número entre 1 e 3.", "red"))
            except ValueError:
                print(colored("Opção inválida. Digite um número.", "red"))

        niveis = ["iniciante", "intermediário", "especialista"]
        nivel = niveis[nivel - 1]
        introducao = f"Ok, vamos lá! Sou um {nivel} em {tema}. "

        if nivel == "iniciante":
            introducao += "Explique os conceitos básicos com exemplos, por favor."
        elif nivel == "intermediário":
            introducao += "Aprofunde as explicações e mostre as relações com outras áreas."
        elif nivel == "especialista":
            introducao += "Compartilhe insights sobre os desafios do tema e discuta diferentes abordagens."

        chat = genai.ChatSession(model=model)
        chat.send_message(introducao)

        while True:
            print(colored("\n--- Faça sua pergunta ---\n", "blue", attrs=["bold"]))
            duvida = input("Sua pergunta: ")
            if duvida.strip().lower() == 'fim':
                print(colored("\n--- Fim da conversa ---\n", "blue", attrs=["bold"]))
                print(colored("Foi um prazer conversar com você! 😊 Até a próxima.", "cyan"))
                return  # Encerra a sessão

            response = chat.send_message(duvida)
            print(colored("\nResposta da IA:\n", "blue"), response.text)

            print(colored("\n--- O que você deseja fazer agora? ---\n", "blue", attrs=["bold"]))
            print(colored("1 - Fazer outra pergunta ❓", "yellow"))
            print(colored("2 - Testar meus conhecimentos com um quiz 🧠", "green"))
            print(colored("3 - Escolher um novo tema 💡", "blue"))
            print(colored("4 - Finalizar a conversa 👋", "red"))

            while True:
                try:
                    opcao = int(input("Escolha uma opção (1-4): "))
                    if opcao in range(1, 5):
                        break
                    else:
                        print(colored("Opção inválida. Escolha um número entre 1 e 4.", "red"))
                except ValueError:
                    print(colored("Opção inválida. Digite um número.", "red"))

            if opcao == 1:
                continue  # Continua no mesmo assunto
            elif opcao == 2:
                fazer_quiz(tema, chat, duvida)
                break  # Sai do loop de perguntas após o quiz
            elif opcao == 3:
                conversar_com_especialista_IA()
                return  # Encerra a sessão após reiniciar
            elif opcao == 4:
                print(colored("\n--- Fim da conversa ---\n", "blue", attrs=["bold"]))
                print(colored("Foi um prazer conversar com você! 😊 Até a próxima.", "cyan"))
                return  # Encerra a sessão

    except Exception as e:
        print(colored("\n--- Oops! Algo deu errado ---\n", "red", attrs=["bold"]))
        print(colored("Parece que encontramos um problema. Não se preocupe, estamos trabalhando para resolver! 😊 Por favor, tente novamente mais tarde.", "red"))
        print(colored(f"Detalhes do erro (para nerds): {e}", "red"))

def fazer_quiz(tema, chat, duvida):
    """
    Cria e administra um quiz com perguntas sobre o tema, uma de cada vez.
    Args:
        tema: O tema do quiz.
        chat: A sessão de chat com o modelo Gemini.
        duvida: A dúvida específica do usuário.
    """
    try:
        print(colored("\n## QUIZ ##\n", "green", attrs=["bold"]))
        print(colored(f"Que tal testar seus conhecimentos sobre {tema} com um quiz rápido? 🧠\n", "green"))

        while True:
            num_perguntas = 0
            while True:
                num_perguntas += 1
                prompt_quiz = f"Crie uma pergunta de múltipla escolha sobre {tema}, focando em '{duvida}', com 4 opções de resposta (A, B, C, D), sem mostrar a resposta correta."
                response = chat.send_message(prompt_quiz)
                print(colored(f"\nPergunta {num_perguntas}:\n", "blue"))
                print(response.text)
                resposta_usuario = input("Qual sua resposta (A, B, C ou D)? ").upper()
                prompt_resposta = f"A resposta correta para a pergunta '{response.text}' é a '{resposta_usuario}'? Por que?"
                print(colored("\nResposta:\n", "blue"), chat.send_message(prompt_resposta).text)

                resposta_continuar = input(colored("\nQuer tentar outra pergunta (sim/não)? 🤔\n", "cyan")).lower()
                if resposta_continuar != "sim":
                    break

            print(colored("\n--- Fim do Quiz ---\n", "green", attrs=["bold"]))
            print(colored("Espero que tenha se saído bem! 🎉\n", "green"))
            break

    except Exception as e:
        print(colored("\n--- Oops! Algo deu errado ---\n", "red", attrs=["bold"]))
        print(colored("Parece que encontramos um problema no quiz. Não se preocupe, estamos trabalhando para resolver! 😊 Por favor, tente novamente mais tarde.", "red"))
        print(colored(f"Detalhes do erro (para nerds): {e}", "red"))

# Inicia a conversa
conversar_com_especialista_IA()


-- Bem-vindo ao Chat, sou o professor IAn! --
Olá! 👋 Sou seu especialista em IA. Vamos conversar sobre o que você quiser.

